# Multiple Inheritance
In Python is **not** much more complex than single inheritance. 

Remember to use **isinstance() and issubclass()**

In [2]:
help(isinstance)

Help on built-in function isinstance in module builtins:

isinstance(obj, class_or_tuple, /)
    Return whether an object is an instance of a class or of a subclass thereof.
    
    A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
    or ...`` etc.



In [3]:
# Test it
isinstance(3, int)

True

In [4]:
isinstance("hello", str)

True

In [6]:
isinstance(4.763, bytes)

False

In [7]:
# isinstance accepts tuple of types for the
# second argument
x = []
isinstance(x, (float, dict, list))

True

In [8]:
class SimpleList:
    """
    Sorted list example
    """
    def __init__(self, items):
        self._items = items
    
    def add(self, item):
        self._items.append(item)
        
    def __getitem__(self, index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return "SimpleList({!r})".format(self._items)

In [9]:
class SortedList(SimpleList):
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
    
    # Overwrite method
    def add(self, item):
        super().add(item)
        self.sort()
    
    def __repr__(self):
        return "SortedList({!r})".format(list(self))

In [10]:
# Test it
sl = SortedList([4, 5, 2, 99, 11])
sl

SortedList([2, 4, 5, 11, 99])

In [11]:
len(sl)

5

In [12]:
sl.add(-6)
sl

SortedList([-6, 2, 4, 5, 11, 99])

In [13]:
sl.add(22)
sl

SortedList([-6, 2, 4, 5, 11, 22, 99])

In [14]:
class IntList(SimpleList):
    # call init on every item in the list
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
        
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError("IntList only supports integer values.")
            
    def add(self, item):
        """
        Validate the item before you add it to the list
        """
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return "IntList({!r})".format(list(self))
    

In [16]:
# Test it
il = IntList([1, 2, 8, 9])
il.add(29)
il

IntList([1, 2, 8, 9, 29])

In [17]:
il2 = IntList([1, 5, "yes"])

TypeError: IntList only supports integer values.

### issubclass()
> help(issubclass())

In [18]:
issubclass(IntList, SimpleList)

True

In [19]:
issubclass(SortedList, IntList)

False

## Multiple Inheritance
Define a class with more than one base class<br>
This is not a universal functionality of OO languages. For example C++ does supports it, and Java does not. 

Note: It could lead to some complicated situations. 
- What if more than one base class defines a particular method

Python hs a relative simple and undertandable system for **multiple inheritance**

Syntax: **class SubClass(Base1, Base2, ...)**
- Subclasses inherit methods from ALL bases
- without conflict, names resolve in the obvious way.
- **Method Resolution Order (MRO)** determines name lookup in all cases. 

In [20]:
class SortedIntList(IntList, SortedList):
    def __repr__(self):
        return "SortedIntList({!r})".format(list(self))

In [21]:
# test it
sil = SortedIntList([42, 2, 17, 9])
sil

SortedIntList([2, 9, 17, 42])

In [22]:
sil = SortedIntList([42, 2, 17, "No"])
sil

TypeError: IntList only supports integer values.

In [23]:
# Add method maintains both, the sorting and type constrains
sil.add(-12)
sil

SortedIntList([-12, 2, 9, 17, 42])

In [24]:
sil.add("Weber")

TypeError: IntList only supports integer values.

**How does Python knows which add() to call?**<br>
**How does Python mantains both constrains?**<br>

The answer to both of these questions is, the **resolution order**. 

**MRO and super()** how they work. 

If a class
- has **multiple** base classes
- defines **no initializer** then **only** the initializer of the **first** base class is automatically called. 

In [25]:
class Base1:
    def __init__(self):
        print('Base1.__init__')
        
class Base2:
    def __init__(self):
        print('Base2.__init__')
        
class Sub(Base1, Base2):
    pass

In [26]:
s = Sub()

Base1.__init__


Through the use of **super()** we could design these classes such that both Base1 and Base2 are called automatically. 

**\_\_bases\_\_**: A tuple of base classes

In [27]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

In [28]:
IntList.__bases__

(__main__.SimpleList,)

## Method Resolution Order (MRO)
Ordering that determines method name lookup
- Methods may be defined in multiple places
- MRO is an ordering of the inheritance graph
- It is quite simple!

Let's look at an example, but let's look at where our MRO is stored.
**\_\_mro\_\_**

In [29]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [30]:
# Get it in list form
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

## How is MRO used?
**obj.method()**
class SomeClass
1. instance of:
    - Base1!
    - Base2!
    - Base3! Yes. A hit here
    - Base4
2. MRO
    - match!
    - Base3.method(obj)
3. resolves to
    - ex: 19

In [31]:
class A:
    def func(self):
        return "A.func"

class B(A):
    def func(self):
        return "B.func"
    
class C(A):
    def func(self):
        return "C.func"

class D(C, B):
    pass

In [32]:
# Test it
D.mro()

[__main__.D, __main__.C, __main__.B, __main__.A, object]

In [33]:
# What should you expect when typing d.func()
d = D()
d.func()

'C.func'

In [34]:
# Change the order of base classes
class D(B, C):
    pass

In [35]:
d = D()
d.func()

'B.func'

In [36]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

This is properly maintaining both, ther sorting constraint and the type constraint of both SortredList

**How is IntList.add() deferring to SortedList.add()?**
The answer is how **super()** actually works. 

### How does Python calculates the MRO?

C3: Algorithm for calculating MRO in Python
- Subclasses come **before** base classes.
- Base class order from class definition is **preserved**.
- Firt two qualities are preserved **no matter** where you start in the inheritance graph.

Note: Not all inheritance declarations are allowed

In [37]:
class A:
    pass

class B(A):
    pass

class C(A):
    pass

class D(B, A, C):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

Note: Since both, B and C inherit from A, they **must** come before A. 

C3 cannot put A both before and after C. 

## The built-in super() function
TODO: (come back to it)